# Semantic segmentation
In the previous step we have been able to build a model to classify the CT scan samples whether they are normal (negative) or nodules (positive). We need also to know their size and where they are located. This is a complete different task called segmentation. Segmentation is a task aimed at classifying every pixel in an image. The output expected from a segmentation task of an input 3D image is a raster image in which each voxel is assigned to a class, e.g. represented by a color depending on whether it is part of a normal sample or a nodule. The output of this task will be used to classify the nodules as benign or malignant. For this task we need a different architecture from that used for classification and we will also use a different cost function. The model architecture is [U-Net](https://arxiv.org/abs/1505.04597) that was developed exactly for the segmentation of biomedical images. In order to implement the segmentation task we have to modify the model, the dataset, and the training loop. 

## The U-Net model
The model used for the classification task was the LunaModel, a sequence on blocks, called LunaBlock, containing two convolutional layers with a ReLu activation function and a max pooling layer after each convolution. The output of the LunaModel was a probability for an image to contain a normal sample or a nodule. Now we need a model that can tell the probability of any single voxel to belong to a normal sample or to a nodule. Instead of implementing a new model from scratch we use an [implementation](https://github.com/jvanvugt/pytorch-unet) of the U-Net architecture for PyTorch available online. The U-Net model architecture can be seen in the code. It is made up of two sequences of blocks. A first sequence of UNetConvBlock for the downsampling and a following sequence of UNetUpBlock for the upsampling. The length of the two sequences can be set depending on the size of the input. We add some customization to the U-Net model that is implemented in a wrapper, the UNetWrapper.   

## The dataset
We use 2D 512x512 images as input for our wrapper of the U-Net model instead of a 3D image because U-Net uses 64 channels in the 1st layer that would result in a memory usage of 512x512x128x2x64 = 8 GB already at the 1st convolutional layer of the 1st block that would become 20 GB just at the end of the 1st block, whith H = W = 512, B = 128 and C = 64 and H height, W width, B batch size, C = channels. One other thing that we need to change are the labels of the dataset. The original labels were used to locate the nodules but not their size or shape so they cannot be used directly to train and validate the performance of the new model to segment a nodule. We have to convert the location of the nodules into a bounding box. We assume a location is in the centre of mass of the nodule and from it we can find out its size along the three spatial dimensions including in the nodule bounding box all the voxels whose density is above a threshold. The bounding box of the nodules provides our annotation mask that can be used as labels for the nodules in the dataset for the training and validation steps.     

## The training dataset
We will set up two datasets: one for training, TrainingLuna2dSegmentationDataset, and one for validation, Luna2dSegmentationDataset. The training dataset is randomized and instead of using the full slice whose size is 512x512 we will use a cropped version of size 64x64. For each nodule we will create different crops around each of them. We want to apply the data augmentation technique to our training dataset. The augmentation is performed in the GPU so the data is first moved from the CPU to the GPU where the techinique is applied. We use a customized model to perform the augmentation

## The training script
As said in the introduction we are going to use a different optimizer, replacing the Stochastic Gradient Descent, and a different cost function. The optimizer we use is Adam and the loss is called Dice. Adam is an optimizer that choose the best learning step for the data at hand. The Dice loss works like the F1 score that we used for the classification task, the diffrence here is that we use the same metrics for each pixel. The loss is computed by counting the pixels in a slice that have been missclassified. We will give more weight to the positive sample in order to counterbalance the higher number of negative ones. Doing so will result in a higher recall but that is what we want since it's very important to not miss a nodule that might be malignant.

In [ ]:
!git clone https://github.com/deep-learning-with-pytorch/dlwpt-code.git

Cloning into 'dlwpt-code'...
remote: Enumerating objects: 703, done.
remote: Total 703 (delta 0), reused 0 (delta 0), pack-reused 703
Receiving objects: 100% (703/703), 176.00 MiB | 9.23 MiB/s, done.
Resolving deltas: 100% (309/309), done.
Checking out files: 100% (228/228), done.


## Downloading the data

In [ ]:
cd dlwpt-code/

/content/dlwpt-code


In [ ]:
import os
os.makedirs('data-unversioned/part2/luna')

In [ ]:
cd data-unversioned/part2/luna

/content/dlwpt-code/data-unversioned/part2/luna


In [ ]:
!wget https://zenodo.org/record/3723295/files/subset0.zip

--2022-12-07 09:52:11--  https://zenodo.org/record/3723295/files/subset0.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6811924508 (6.3G) [application/octet-stream]
Saving to: ‘subset0.zip’

subset0.zip         100%[===================>]   6.34G  28.2MB/s    in 13m 16s 

2022-12-07 10:05:28 (8.16 MB/s) - ‘subset0.zip’ saved [6811924508/6811924508]



In [ ]:
!7z x subset0.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 6811924508 bytes (6497 MiB)

Extracting archive: subset0.zip

ERRORS:
Headers Error

--
Path = subset0.zip
Type = zip
ERRORS:
Headers Error
Physical Size = 6811924508
64-bit = +

  0%      0% 1 - subset0/1.3.6.1.4.1.14519.5.2.1.6 . 105756658031515062000744821260.raw                                                                                 0% 2        0% 3 - subset0/1.3.6.1.4.1.14519.5.2.1.6 . 108197895896446896160048741492.raw                                

In [ ]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 228 kB/s 


In [ ]:
!pip install "diskcache==4.1.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.1 MB/s 


In [ ]:
cd /content/dlwpt-code/

/content/dlwpt-code


## Setting up the training and validation datasets
Use the notebook p2ch13_explore_data_v2.ipynb  
Implement the model_seg.py module or uncomment the code for SegmentaionMask and MaskTuple in module.py. Follow the comments for the issue 68

https://github.com/deep-learning-with-pytorch/dlwpt-code/issues/68

In [ ]:
%matplotlib inline
import copy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import torch
from p2ch13.dsets import getCandidateInfoList, getCt, TrainingLuna2dSegmentationDataset, CandidateInfoTuple #, old_build2dLungMask
from p2ch13.model_seg import SegmentationMask, MaskTuple
from p2ch13.vis import build2dLungMask
from util.util import xyz2irc

In [ ]:
candidateInfo_list = getCandidateInfoList(requireOnDisk_bool=False)
candidateInfo_list[0]

CandidateInfoTuple(isNodule_bool=True, hasAnnotation_bool=True, isMal_bool=True, diameter_mm=32.27003025, series_uid='1.3.6.1.4.1.14519.5.2.1.6279.6001.287966244644280690737019247886', center_xyz=(67.82725575, 85.37992457, -109.74672379999998))

In [ ]:
series_list = sorted(set(t.series_uid for t in candidateInfo_list))

In [ ]:
def transparent_cmap(cmap, N=255):
    "Copy colormap and set alpha values"

    mycmap = copy.deepcopy(cmap)
    mycmap._init()
    mycmap._lut[:,-1] = np.linspace(0, 0.75, N+4)
    return mycmap
tgray = transparent_cmap(plt.cm.gray)
tpurp = transparent_cmap(plt.cm.Purples)
tblue = transparent_cmap(plt.cm.Blues)
tgreen = transparent_cmap(plt.cm.Greens)
torange = transparent_cmap(plt.cm.Oranges)
tred = transparent_cmap(plt.cm.Reds)


clim=(0, 1.3)
start_ndx = 3
mask_model = SegmentationMask()

In [ ]:
ct_list = []
for nit_ndx in range(start_ndx, start_ndx+3):
    candidateInfo_tup = candidateInfo_list[nit_ndx]
    ct = getCt(candidateInfo_tup.series_uid)
    center_irc = xyz2irc(candidateInfo_tup.center_xyz, ct.origin_xyz, ct.vxSize_xyz, ct.direction_a)
    
    ct_list.append((ct, center_irc))
start_ndx = nit_ndx + 1

fig = plt.figure(figsize=(60,90))
subplot_ndx = 0 
for ct_ndx, (ct, center_irc) in enumerate(ct_list):
    mask_tup = build2dLungMask(ct.series_uid, int(center_irc.index))
    old_tup = old_build2dLungMask(ct.series_uid, int(center_irc.index))
    
#    ct_g = torch.from_numpy(ct.hu_a[int(center_irc.index)].astype(np.float32)).unsqueeze(0).unsqueeze(0).to('cuda')
#    pos_g = torch.from_numpy(ct.positive_mask[int(center_irc.index)].astype(np.float32)).unsqueeze(0).unsqueeze(0).to('cuda')
#    input_g = ct_g / 1000
    
#    label_g, neg_g, pos_g, lung_mask, mask_dict = mask_model(input_g, pos_g)
#    mask_tup = MaskTuple(**mask_dict)
    for attr_ndx, attr_str in enumerate(mask_tup._fields):

        subplot_ndx = 1 + 3 * 2 * attr_ndx + 2 * ct_ndx
        subplot = fig.add_subplot(len(mask_tup), len(ct_list)*2, subplot_ndx)
        subplot.set_title(attr_str)
        
        
        #print(layer_func, ct.hu_a.shape, layer_func(ct, mask_tup, int(center_irc.index)).shape, center_irc.index)

        plt.imshow(ct.hu_a[int(center_irc.index)], clim=(-1000, 3000), cmap='RdGy')
        plt.imshow(mask_tup[attr_ndx][0][0].cpu(), clim=clim, cmap=tblue)

        subplot = fig.add_subplot(len(mask_tup), len(ct_list)*2, subplot_ndx+1)
        subplot.set_title('old '+ attr_str)

        plt.imshow(ct.hu_a[int(center_irc.index)], clim=(-1000, 3000), cmap='RdGy')
        plt.imshow(old_tup[attr_ndx], clim=clim, cmap=tblue)
        
        #if attr_ndx == 1: break
    #break


IndexError: ignored